# Debugging simple program from jupyter Notebook

This notebook shows how to control a GDB session from a jupyter notebook

Given a trivial c-program:

```c
// file: dummy.c
#include <stdio.h>

int main()
{
   int i = 0;

   for(i=0; i<10; i++)
       printf("i: %d\n",i);

   return 0;
}
```

Which I compile with debug flags:

```shell
gcc dummy.c -g -o dummy
```

I can create a `gdbserver` session (on the same machine where jupyter is running) which I can control through my jupyter notebook

```shell
$ gdbserver 0.0.0.0:1234 dummy
Process dummy created; pid = 3610
Listening on port 1234
```

Let us first connect to the `gdbserver` on our localhost (`127.0.0.1`)

In [1]:
target remote 127.0.0.1:1234

 Remote debugging using 127.0.0.1:1234
Reading /home/vagrant/jupyter-gdb-kernel/test/dummy from remote target...
Reading /home/vagrant/jupyter-gdb-kernel/test/dummy from remote target...
Reading symbols from target:/home/vagrant/jupyter-gdb-kernel/test/dummy...done.
Reading /lib64/ld-linux-x86-64.so.2 from remote target...
Reading /lib64/ld-linux-x86-64.so.2 from remote target...
Reading symbols from target:/lib64/ld-linux-x86-64.so.2...Reading /lib64/ld-2.23.so from remote target...
Reading /lib64/.debug/ld-2.23.so from remote target...
(no debugging symbols found)...done.
0x00007ffff7dd7cc0 in ?? () from target:/lib64/ld-linux-x86-64.so.2


: 1

We can show the source code using the `list` command

In [2]:
list main

 1	#include <stdio.h>
2	
3	
4	int main()
5	{
6	    int i = 0;
7	
8	    for(i=0; i<10; i++)
9	        printf("i: %d\n",i);
10	


: 1

Let us set a first breakpoint at line 6

In [3]:
break 6

 Breakpoint 1 at 0x40052e: file dummy.c, line 6.


: 1

We can now start the dummy program using `continue`, it will continue executuin and then halt at our breakpoint

In [4]:
continue

 Continuing.
Reading /lib/x86_64-linux-gnu/libc.so.6 from remote target...
Reading /lib/x86_64-linux-gnu/libc-2.23.so from remote target...
Reading /lib/x86_64-linux-gnu/.debug/libc-2.23.so from remote target...

Breakpoint 1, main () at dummy.c:6
6	    int i = 0;


: 1

Lets inspect all the local variables here using `info locals`

In [5]:
info locals

 i = 0


: 1

It is only `i` for this trivial program, we can set a conditional breakpoint at line `9` to stop when i reaches `7`

In [6]:
break 9 if (i == 7)

 Breakpoint 2 at 0x40053e: file dummy.c, line 9.


: 1

Since the execution is still halted, we will need to `continue` it again

In [7]:
continue

 Continuing.

Breakpoint 2, main () at dummy.c:9
9	        printf("i: %d\n",i);


: 1

Lets check if `i` has the expected value using `printf`

In [8]:
printf "The value of i is %d\n", i

 The value of i is 7


: 1

If we take a look at all our breakpoints using `info breakpoints`

In [9]:
info breakpoints

 Num     Type           Disp Enb Address            What
1       breakpoint     keep y   0x000000000040052e in main at dummy.c:6
	breakpoint already hit 1 time
2       breakpoint     keep y   0x000000000040053e in main at dummy.c:9
	stop only if (i == 7) (target evals)
	breakpoint already hit 1 time


: 1

We can see that the breakpoint will only trigger when `i == 7` so if we `continue` the program will finish

In [10]:
continue

 Continuing.
[Inferior 1 (process 3610) exited normally]


: 1